# All used features for the three regression model implementations:

1. delta_Q<sub>100-10</sub>(V)
    - Minimum
    - Vairance
    - Skewness
    - Kurtosis
    
    
2. Discharge capacity fade curve features
    - Slope of the linear fit to the capacity fade curve, cycles 2 to 100
    - Intercept of the linear fit to capavity face curve, cycles 2 to 100
    - Discharge capacity, cycle 2
    - Difference between max discharge capacity and cycle 2
    
    
3. Other features
    - Average charge time, first 5 cycles
    - Integral of temperature over time, cycles 2 to 100
    - Minimum internal resistance, difference between cycle 100 and cycle 2
    
    
4. Multivariate discharge curves features
    - Slope of capacity fade curve, cycles 2 to 100/300
    - Intercept of the linear fit to capacity fade curve, cycles 2 to 100/300
    - Slope of capacity fade curve, cycles 91 to 100 or 291 to 300
    - Intercept of the linear fit to the capacity fade curve, cycles 91 to 100 or 291 to 300
    - Discharge capacity, cycle 2
    - Max discharge capacity – discharge capacity, cycle 2
    - Discharge capacity, cycle 100/300

In [ ]:
# Replicating Load Data logic

import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import pickle
from pathlib import Path

In [ ]:
path1 = Path("Data/batch1.pkl")
batch1 = pickle.load(open(path1, 'rb'))

#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

numBat1 = len(batch1.keys())
numBat1

In [ ]:
features_df = pd.DataFrame()

### 1. delta_Q<sub>100-10</sub>(V)

In [ ]:
minimum_dQ_100_10 = np.zeros(len(batch1.keys()))
variance_dQ_100_10 = np.zeros(len(batch1.keys()))
skewness_dQ_100_10 = np.zeros(len(batch1.keys()))
kurtosis_dQ_100_10 = np.zeros(len(batch1.keys()))

for i, cell in enumerate(batch1.values()):
    c10 = cell['cycles']['10']
    c100 = cell['cycles']['100']
    dQ_100_10 = c100['Qdlin'] - c10['Qdlin']
    
    minimum_dQ_100_10[i] = np.log(np.abs(np.min(dQ_100_10)))
    variance_dQ_100_10[i] = np.log(np.var(dQ_100_10))
    skewness_dQ_100_10[i] = np.log(np.abs(skew(dQ_100_10)))
    kurtosis_dQ_100_10[i] = np.log(np.abs(kurtosis(dQ_100_10)))
    
df = pd.DataFrame({
    "minimum_dQ_100_10": minimum_dQ_100_10, 
    "variance_dQ_100_10": variance_dQ_100_10,
    "skewness_dQ_100_10": skewness_dQ_100_10,
    "kurtosis_dQ_100_10": kurtosis_dQ_100_10
})

df.head()